# Bank Customer Churn Analysis - Exploratory Data Analysis

This notebook provides comprehensive exploratory data analysis for the bank customer churn dataset.

## Objectives
- Understand data distributions and patterns
- Analyze churn rates across different segments
- Identify key relationships and correlations
- Generate insights for feature engineering and modeling

## Table of Contents
1. [Data Loading and Overview](#1-data-loading-and-overview)
2. [Univariate Analysis](#2-univariate-analysis)
3. [Bivariate Analysis](#3-bivariate-analysis)
4. [Churn Analysis](#4-churn-analysis)
5. [Correlation Analysis](#5-correlation-analysis)
6. [Geographic Analysis](#6-geographic-analysis)
7. [Customer Segmentation Insights](#7-customer-segmentation-insights)
8. [Key Findings and Recommendations](#8-key-findings-and-recommendations)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import sys

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
warnings.filterwarnings('ignore')

# Set up paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
reports_dir = project_root / 'reports' / 'figures'
reports_dir.mkdir(parents=True, exist_ok=True)

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Data Loading and Overview

In [ ]:
# Load the processed data
try:
    df = pd.read_parquet(data_dir / 'processed' / 'churn_cleaned.parquet')
    print(f'✅ Loaded processed data: {df.shape}')
except FileNotFoundError:
    try:
        df = pd.read_parquet(data_dir / 'interim' / 'churn_raw.parquet')
        print(f'⚠️  Loaded interim data: {df.shape}')
        print('Note: Using interim data. Run data cleaning pipeline for processed data.')
    except FileNotFoundError:
        print('❌ No data found. Please run the data loading pipeline first.')
        sys.exit(1)

# Display basic information
print(f'\nDataset Shape: {df.shape}')
print(f'Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

# Display first few rows
df.head()

In [ ]:
# Data types and missing values
print('Data Types and Missing Values:')
print('=' * 50)

info_df = pd.DataFrame({
    'Data Type': df.dtypes,
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100,
    'Unique Values': df.nunique(),
    'Sample Values': [str(df[col].dropna().iloc[:3].tolist()) for col in df.columns]
})

info_df

In [ ]:
# Statistical summary
print('Statistical Summary:')
print('=' * 50)
df.describe(include='all').round(2)

## 2. Univariate Analysis

In [ ]:
# Target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Churn distribution
churn_counts = df['Exited'].value_counts()
churn_pct = df['Exited'].value_counts(normalize=True) * 100

axes[0].pie(churn_counts.values, labels=['Retained', 'Churned'], autopct='%1.1f%%', startangle=90)
axes[0].set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')

# Bar plot
sns.countplot(data=df, x='Exited', ax=axes[1])
axes[1].set_title('Churn Count Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Exited (0=Retained, 1=Churned)')

# Add count labels
for i, v in enumerate(churn_counts.values):
    axes[1].text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig(reports_dir / 'churn_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f'Churn Rate: {churn_pct[1]:.2f}%')
print(f'Retention Rate: {churn_pct[0]:.2f}%')

In [ ]:
# Numeric features distributions
numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    if col in df.columns:
        # Histogram with KDE
        sns.histplot(data=df, x=col, kde=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}', fontweight='bold')
        axes[i].axvline(df[col].mean(), color='red', linestyle='--', alpha=0.7, label=f'Mean: {df[col].mean():.1f}')
        axes[i].axvline(df[col].median(), color='green', linestyle='--', alpha=0.7, label=f'Median: {df[col].median():.1f}')
        axes[i].legend()

# Remove empty subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig(reports_dir / 'numeric_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Categorical features distributions
categorical_cols = ['Geography', 'Gender', 'NumOfProducts', 'HasCrCard', 'IsActiveMember']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, col in enumerate(categorical_cols):
    if col in df.columns:
        value_counts = df[col].value_counts()
        sns.countplot(data=df, x=col, ax=axes[i], order=value_counts.index)
        axes[i].set_title(f'Distribution of {col}', fontweight='bold')
        
        # Add percentage labels
        total = len(df)
        for p in axes[i].patches:
            percentage = f'{100 * p.get_height() / total:.1f}%'
            axes[i].annotate(percentage, (p.get_x() + p.get_width()/2., p.get_height()),
                           ha='center', va='bottom', fontweight='bold')
        
        # Rotate x-axis labels if needed
        if col == 'Geography':
            axes[i].tick_params(axis='x', rotation=45)

# Remove empty subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig(reports_dir / 'categorical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Bivariate Analysis

In [ ]:
# Age vs Balance by Churn Status
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
for exit_status in [0, 1]:
    subset = df[df['Exited'] == exit_status]
    label = 'Churned' if exit_status == 1 else 'Retained'
    axes[0].scatter(subset['Age'], subset['Balance'], alpha=0.6, label=label)

axes[0].set_xlabel('Age')
axes[0].set_ylabel('Balance')
axes[0].set_title('Age vs Balance by Churn Status', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot of Age by Churn
sns.boxplot(data=df, x='Exited', y='Age', ax=axes[1])
axes[1].set_title('Age Distribution by Churn Status', fontweight='bold')
axes[1].set_xlabel('Exited (0=Retained, 1=Churned)')

plt.tight_layout()
plt.savefig(reports_dir / 'age_balance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Credit Score and Estimated Salary Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Credit Score by Churn
sns.boxplot(data=df, x='Exited', y='CreditScore', ax=axes[0,0])
axes[0,0].set_title('Credit Score by Churn Status', fontweight='bold')

# Estimated Salary by Churn
sns.boxplot(data=df, x='Exited', y='EstimatedSalary', ax=axes[0,1])
axes[0,1].set_title('Estimated Salary by Churn Status', fontweight='bold')

# Balance by Churn
sns.boxplot(data=df, x='Exited', y='Balance', ax=axes[1,0])
axes[1,0].set_title('Balance by Churn Status', fontweight='bold')

# Tenure by Churn
sns.boxplot(data=df, x='Exited', y='Tenure', ax=axes[1,1])
axes[1,1].set_title('Tenure by Churn Status', fontweight='bold')

plt.tight_layout()
plt.savefig(reports_dir / 'numeric_features_by_churn.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Churn Analysis by Categories

In [ ]:
# Churn rate by categorical features
categorical_features = ['Geography', 'Gender', 'NumOfProducts', 'HasCrCard', 'IsActiveMember']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

churn_rates = {}

for i, feature in enumerate(categorical_features):
    if feature in df.columns:
        # Calculate churn rate by category
        churn_by_category = df.groupby(feature)['Exited'].agg(['count', 'sum', 'mean']).round(3)
        churn_by_category['churn_rate'] = churn_by_category['mean'] * 100
        churn_rates[feature] = churn_by_category
        
        # Plot
        bars = axes[i].bar(churn_by_category.index.astype(str), churn_by_category['churn_rate'])
        axes[i].set_title(f'Churn Rate by {feature}', fontweight='bold')
        axes[i].set_ylabel('Churn Rate (%)')
        axes[i].set_xlabel(feature)
        
        # Add value labels on bars
        for bar, rate in zip(bars, churn_by_category['churn_rate']):
            axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                        f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        # Rotate x-axis labels if needed
        if feature in ['Geography']:
            axes[i].tick_params(axis='x', rotation=45)

# Remove empty subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig(reports_dir / 'churn_rates_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

# Print detailed churn rates
print('Detailed Churn Rates by Category:')
print('=' * 50)
for feature, rates in churn_rates.items():
    print(f'\n{feature}:')
    print(rates[['count', 'sum', 'churn_rate']])

In [ ]:
# Age group analysis
# Create age groups
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 30, 40, 50, 60, 100], 
                       labels=['18-30', '31-40', '41-50', '51-60', '60+'])

# Churn rate by age group
age_churn = df.groupby('AgeGroup')['Exited'].agg(['count', 'sum', 'mean']).round(3)
age_churn['churn_rate'] = age_churn['mean'] * 100

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot
bars = axes[0].bar(age_churn.index.astype(str), age_churn['churn_rate'])
axes[0].set_title('Churn Rate by Age Group', fontweight='bold')
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_xlabel('Age Group')

# Add value labels
for bar, rate in zip(bars, age_churn['churn_rate']):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')

# Stacked bar chart
age_counts = df.groupby(['AgeGroup', 'Exited']).size().unstack(fill_value=0)
age_counts.plot(kind='bar', stacked=True, ax=axes[1], color=['lightblue', 'salmon'])
axes[1].set_title('Customer Count by Age Group and Churn Status', fontweight='bold')
axes[1].set_ylabel('Number of Customers')
axes[1].set_xlabel('Age Group')
axes[1].legend(['Retained', 'Churned'])
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(reports_dir / 'age_group_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print('Churn Rate by Age Group:')
print(age_churn[['count', 'sum', 'churn_rate']])

## 5. Correlation Analysis

In [ ]:
# Correlation heatmap of numeric features
numeric_features = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_features].corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')
plt.title('Correlation Heatmap of Numeric Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(reports_dir / 'correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# Find strongest correlations with target variable
target_correlations = correlation_matrix['Exited'].abs().sort_values(ascending=False)
print('Strongest Correlations with Churn (Exited):')
print('=' * 45)
for feature, corr in target_correlations.items():
    if feature != 'Exited':
        print(f'{feature:<20}: {corr:.3f}')

In [ ]:
# Feature relationships with target
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Age vs Churn
sns.violinplot(data=df, x='Exited', y='Age', ax=axes[0,0])
axes[0,0].set_title('Age Distribution by Churn Status', fontweight='bold')

# Number of Products vs Churn
product_churn = df.groupby('NumOfProducts')['Exited'].mean() * 100
bars = axes[0,1].bar(product_churn.index, product_churn.values)
axes[0,1].set_title('Churn Rate by Number of Products', fontweight='bold')
axes[0,1].set_xlabel('Number of Products')
axes[0,1].set_ylabel('Churn Rate (%)')
for bar, rate in zip(bars, product_churn.values):
    axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                  f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')

# Balance distribution by churn
df_sample = df.sample(n=min(5000, len(df)), random_state=42)  # Sample for better visualization
for exit_status in [0, 1]:
    subset = df_sample[df_sample['Exited'] == exit_status]
    label = 'Churned' if exit_status == 1 else 'Retained'
    axes[1,0].hist(subset['Balance'], alpha=0.7, bins=30, label=label, density=True)
axes[1,0].set_title('Balance Distribution by Churn Status', fontweight='bold')
axes[1,0].set_xlabel('Balance')
axes[1,0].set_ylabel('Density')
axes[1,0].legend()

# Geography and Gender interaction
geo_gender_churn = df.groupby(['Geography', 'Gender'])['Exited'].mean().unstack() * 100
sns.heatmap(geo_gender_churn, annot=True, fmt='.1f', cmap='Reds', ax=axes[1,1])
axes[1,1].set_title('Churn Rate by Geography and Gender (%)', fontweight='bold')

plt.tight_layout()
plt.savefig(reports_dir / 'feature_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Geographic Analysis

In [ ]:
# Geographic analysis
geo_analysis = df.groupby('Geography').agg({
    'Exited': ['count', 'sum', 'mean'],
    'Age': 'mean',
    'CreditScore': 'mean',
    'Balance': 'mean',
    'EstimatedSalary': 'mean'
}).round(2)

# Flatten column names
geo_analysis.columns = ['_'.join(col).strip() for col in geo_analysis.columns]
geo_analysis['churn_rate'] = geo_analysis['Exited_mean'] * 100

print('Geographic Analysis Summary:')
print('=' * 50)
print(geo_analysis)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Churn rate by geography
bars = axes[0,0].bar(geo_analysis.index, geo_analysis['churn_rate'])
axes[0,0].set_title('Churn Rate by Geography', fontweight='bold')
axes[0,0].set_ylabel('Churn Rate (%)')
for bar, rate in zip(bars, geo_analysis['churn_rate']):
    axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                  f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')

# Average metrics by geography
metrics = ['Age_mean', 'CreditScore_mean', 'Balance_mean']
x = np.arange(len(geo_analysis.index))
width = 0.25

for i, metric in enumerate(metrics):
    # Normalize values for comparison
    normalized_values = geo_analysis[metric] / geo_analysis[metric].max()
    axes[0,1].bar(x + i*width, normalized_values, width, label=metric.replace('_mean', ''))

axes[0,1].set_title('Normalized Average Metrics by Geography', fontweight='bold')
axes[0,1].set_xlabel('Geography')
axes[0,1].set_ylabel('Normalized Value')
axes[0,1].set_xticks(x + width)
axes[0,1].set_xticklabels(geo_analysis.index)
axes[0,1].legend()

# Customer distribution by geography
geo_counts = df['Geography'].value_counts()
axes[1,0].pie(geo_counts.values, labels=geo_counts.index, autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('Customer Distribution by Geography', fontweight='bold')

# Churn count by geography
geo_churn_counts = df.groupby(['Geography', 'Exited']).size().unstack(fill_value=0)
geo_churn_counts.plot(kind='bar', ax=axes[1,1], color=['lightblue', 'salmon'])
axes[1,1].set_title('Customer Count by Geography and Churn Status', fontweight='bold')
axes[1,1].set_ylabel('Number of Customers')
axes[1,1].legend(['Retained', 'Churned'])
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(reports_dir / 'geographic_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Customer Segmentation Insights

In [ ]:
# Create customer segments based on key characteristics
# High-value customers
df['HighValue'] = ((df['Balance'] > df['Balance'].quantile(0.75)) | 
                   (df['EstimatedSalary'] > df['EstimatedSalary'].quantile(0.75))).astype(int)

# Active customers
df['HighlyActive'] = ((df['IsActiveMember'] == 1) & 
                      (df['NumOfProducts'] >= 2) & 
                      (df['HasCrCard'] == 1)).astype(int)

# Young customers
df['YoungCustomer'] = (df['Age'] <= 35).astype(int)

# Senior customers
df['SeniorCustomer'] = (df['Age'] >= 55).astype(int)

# Analyze segments
segments = ['HighValue', 'HighlyActive', 'YoungCustomer', 'SeniorCustomer']

segment_analysis = {}
for segment in segments:
    segment_data = df[df[segment] == 1]
    segment_analysis[segment] = {
        'count': len(segment_data),
        'percentage': len(segment_data) / len(df) * 100,
        'churn_rate': segment_data['Exited'].mean() * 100,
        'avg_age': segment_data['Age'].mean(),
        'avg_balance': segment_data['Balance'].mean(),
        'avg_tenure': segment_data['Tenure'].mean()
    }

# Convert to DataFrame for better display
segment_df = pd.DataFrame(segment_analysis).T.round(2)

print('Customer Segment Analysis:')
print('=' * 50)
print(segment_df)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Segment sizes
segment_counts = [segment_analysis[seg]['count'] for seg in segments]
axes[0,0].bar(segments, segment_counts)
axes[0,0].set_title('Customer Segment Sizes', fontweight='bold')
axes[0,0].set_ylabel('Number of Customers')
axes[0,0].tick_params(axis='x', rotation=45)

# Churn rates by segment
churn_rates = [segment_analysis[seg]['churn_rate'] for seg in segments]
bars = axes[0,1].bar(segments, churn_rates, color='salmon')
axes[0,1].set_title('Churn Rate by Customer Segment', fontweight='bold')
axes[0,1].set_ylabel('Churn Rate (%)')
axes[0,1].tick_params(axis='x', rotation=45)
for bar, rate in zip(bars, churn_rates):
    axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                  f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')

# Average balance by segment
avg_balances = [segment_analysis[seg]['avg_balance'] for seg in segments]
axes[1,0].bar(segments, avg_balances, color='lightgreen')
axes[1,0].set_title('Average Balance by Customer Segment', fontweight='bold')
axes[1,0].set_ylabel('Average Balance')
axes[1,0].tick_params(axis='x', rotation=45)

# Average tenure by segment
avg_tenures = [segment_analysis[seg]['avg_tenure'] for seg in segments]
axes[1,1].bar(segments, avg_tenures, color='lightcoral')
axes[1,1].set_title('Average Tenure by Customer Segment', fontweight='bold')
axes[1,1].set_ylabel('Average Tenure (years)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(reports_dir / 'customer_segments.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Key Findings and Recommendations

In [ ]:
# Summary statistics and insights
print('🔍 KEY FINDINGS FROM EXPLORATORY DATA ANALYSIS')
print('=' * 60)

# Overall churn rate
overall_churn = df['Exited'].mean() * 100
print(f'📊 Overall Churn Rate: {overall_churn:.2f}%')

# Key insights
print('\n🎯 TOP CHURN DRIVERS:')
print('-' * 30)

# Age analysis
senior_churn = df[df['Age'] >= 55]['Exited'].mean() * 100
young_churn = df[df['Age'] <= 35]['Exited'].mean() * 100
print(f'👴 Senior customers (55+): {senior_churn:.1f}% churn rate')
print(f'👶 Young customers (≤35): {young_churn:.1f}% churn rate')

# Product analysis
for products in sorted(df['NumOfProducts'].unique()):
    product_churn = df[df['NumOfProducts'] == products]['Exited'].mean() * 100
    print(f'📦 {products} product(s): {product_churn:.1f}% churn rate')

# Geography analysis
print('\n🌍 GEOGRAPHIC INSIGHTS:')
print('-' * 30)
for geo in df['Geography'].unique():
    geo_churn = df[df['Geography'] == geo]['Exited'].mean() * 100
    geo_count = len(df[df['Geography'] == geo])
    print(f'{geo}: {geo_churn:.1f}% churn rate ({geo_count:,} customers)')

# Activity analysis
active_churn = df[df['IsActiveMember'] == 1]['Exited'].mean() * 100
inactive_churn = df[df['IsActiveMember'] == 0]['Exited'].mean() * 100
print('\n💼 ACTIVITY INSIGHTS:')
print('-' * 30)
print(f'Active members: {active_churn:.1f}% churn rate')
print(f'Inactive members: {inactive_churn:.1f}% churn rate')

# Balance insights
zero_balance_churn = df[df['Balance'] == 0]['Exited'].mean() * 100
high_balance_churn = df[df['Balance'] > df['Balance'].quantile(0.75)]['Exited'].mean() * 100
print('\n💰 BALANCE INSIGHTS:')
print('-' * 30)
print(f'Zero balance customers: {zero_balance_churn:.1f}% churn rate')
print(f'High balance customers (top 25%): {high_balance_churn:.1f}% churn rate')

print('\n💡 BUSINESS RECOMMENDATIONS:')
print('=' * 40)
print('1. 🎯 Focus retention efforts on customers with 3-4 products')
print('2. 👥 Develop targeted campaigns for German customers')
print('3. 🔄 Implement engagement programs for inactive members')
print('4. 👴 Create senior-friendly services and support')
print('5. 💳 Encourage credit card adoption among non-holders')
print('6. 📈 Monitor customers with zero balances closely')
print('7. 🏆 Reward long-tenure customers to maintain loyalty')

In [ ]:
# Save key insights to file
insights = {
    'overall_churn_rate': float(overall_churn),
    'total_customers': len(df),
    'churned_customers': int(df['Exited'].sum()),
    'churn_by_geography': df.groupby('Geography')['Exited'].mean().to_dict(),
    'churn_by_products': df.groupby('NumOfProducts')['Exited'].mean().to_dict(),
    'churn_by_activity': {
        'active': float(active_churn),
        'inactive': float(inactive_churn)
    },
    'age_insights': {
        'senior_churn': float(senior_churn),
        'young_churn': float(young_churn)
    },
    'balance_insights': {
        'zero_balance_churn': float(zero_balance_churn),
        'high_balance_churn': float(high_balance_churn)
    }
}

import json
with open(reports_dir.parent / 'eda_insights.json', 'w') as f:
    json.dump(insights, f, indent=2)

print('\n✅ EDA completed successfully!')
print(f'📁 Visualizations saved to: {reports_dir}')
print(f'📄 Insights saved to: {reports_dir.parent / "eda_insights.json"}')